In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 1. Create normalization function

In [2]:
import pandas as pd

def normalize_landmarks(landmarks, anchor_point):
    normalized = []
    for lm in landmarks:
        # Normalize each landmark by subtracting anchor_point's coordinates
        normalized.append((lm.x - anchor_point.x, lm.y - anchor_point.y, lm.z - anchor_point.z))
    return normalized


# 2. Track coordinates on landmarks (through webcam pose tracking)

In [4]:
# Define global variable to store normalized landmarks
normalized_landmarks = None

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
cap = cv.VideoCapture(0)


# Define your anchor point (set to Nose)
anchor_point = None  # We'll set this in the first frame

# Start video capture and pose processing
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Convert image color for MediaPipe processing
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image
        results = pose.process(image)

        # If landmarks are found, normalize them and store in the global variable
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Set the anchor point to nose in the first frame
            if anchor_point is None:
                anchor_point = landmarks[mp_pose.PoseLandmark.NOSE]
            
            # Normalize the landmarks and store them in the global variable
            anchor_point = landmarks[mp_pose.PoseLandmark.NOSE]
            normalized_landmarks = normalize_landmarks(landmarks, anchor_point)

            # Draw landmarks on the image
            image.flags.writeable = True
            image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Show the processed image
        cv.imshow('Pose Tracking', image)
        
        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()

I0000 00:00:1737278589.313728 27220108 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1 Pro
W0000 00:00:1737278589.414621 27221270 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737278589.429256 27221271 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737278589.574073 27221270 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


KeyboardInterrupt: 

# 3. Convert coords to dataframe

In [5]:
coords_df = pd.DataFrame(normalized_landmarks, columns=['x', 'y', 'z'])
coords_df

,x,y,z
0,0.000000,0.000000,0.000000
1,0.052118,-0.073644,0.017149
2,0.075598,-0.068675,0.017273
3,0.096219,-0.061472,0.017463
4,-0.013544,-0.085464,0.042462
5,-0.029906,-0.088433,0.043280
6,-0.043077,-0.090243,0.043098
7,0.141156,-0.020709,0.226222
8,-0.045114,-0.072676,0.355676
9,0.039936,0.103163,0.083517


: 